Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

In [11]:
from sympy import symbols, Function
import sympy.functions as sym
from sympy import init_printing
init_printing(use_latex='latex')
from sympy import pprint
from sympy import Symbol
from sympy import pi

x = Symbol('x')

(pi + x)**2

       2
(x + π) 

In [12]:
pprint((pi + x)**2)



       2
(x + π) 
